# Úvod do Type Hintingu v Pythonu

Type hinting je vlastnost Pythonu, která umožňuje vývojářům poskytovat náznaky očekávaných datových typů argumentů funkcí a návratových hodnot. Tato praxe činí kód čitelnějším a udržovatelnějším a zároveň umožňuje lepší analýzu kódu, automatické dokončování a kontrolu chyb pomocí editorů a linterů.

Zde se seznámíme se základy type hinting v Pythonu:

1. Základní syntaxe
2. Základní type hinty
3. Modul typing
4. Type aliasy
5. Generické typy
6. Type hinty pro NumPy

## 1. Základní syntaxe

Chceme-li definovat očekávané typy argumentů a návratových hodnot funkcí, použijeme tzv. anotace typů. Anotace typů jsou zápisy za názvem proměnné, oddělené dvojtečkou (`:`). Návratový typ se zaznamenává za `->` před dvojtečkou na konci řádku s `def`. 


In [ ]:
def jmeno_funkce(argument1: int, argument2: str) -> int:
    return 0

Stjně můžeme definovat typ u jednotlivých proměnných.

In [ ]:
cislo_pi : float = 3.14
nejaky_text : str = "Ahoj světe!"

Můžeme si linter nastavit, aby nám hlásil chyby, pokud se požadované typy pomocí type hintingu neshodují s typy proměnných, které jsou v kódu použity. Nicméně typy hinting nemá žádný vliv na běh programu, takže pokud se neshodují, program běží stejně.

In [ ]:
jmeno_funkce(cislo_pi, nejaky_text)

## 2. Základní type hinty
Vestavěné typy v Pythonu se dají použít jako type hinty, například:
1. **int**: Celá čísla (integer)
2. **float**: Desetinná čísla (floating-point)
3. **bool**: Boolovské hodnoty (True/False)
4. **str**: Řetězce (string)
5. **bytes**: Bytové řetězce (sequence of bytes)
6. **list**: Seznam (list)
7. **tuple**: N-tice (tuple)
8. **dict**: Slovník (dictionary)
9. **set**: Množina (set)
10. **frozenset**: Neměnná množina (frozen set)
11. **object**: Základní třída pro všechny objekty v Pythonu
12. **type**: Typ objektu
13. **None**: Konstanta reprezentující žádnou hodnotu nebo prázdný stav (NoneType)
14. **complex**: Komplexní čísla
15. **bytearray**: Pole bytů (mutable sequence of bytes)
16. **memoryview**: Objekt memoryview pro práci s pamětovými bloky
17. **range**: Rozsah čísel (immutable sequence of numbers)

In [ ]:
def foo2(bar: list) -> int:
    return 0

promenna : list = [1, 2, 3]
foo2(promenna)

In [ ]:
def foo3(bar: range) -> None:
    for i in bar:
        print(i)

a = range(2)
foo3(a)

Dále můžeme používat vlastní třídy jako type hinty.

In [ ]:
class Trida:
    def __init__(self, promenna: int):
        self.promenna = promenna

    def metoda(self, promenna: int) -> int:
        return self.promenna + promenna
    
def foo4(bar: Trida) -> int:
    return bar.metoda(1)

promenna = Trida(1)
foo4(promenna)

Podívejte se na rozdíl v nápovědě k metodě `metoda` v příkladu níže. V prvním případě je nápověda k metodě `metoda` vytvořena pomocí type hintingu, v druhém případě není jasné, co metoda `metoda` očekává jako argumenty a co vrátí neboť není jasné jaké třídy jsou použity.

In [ ]:
def foo5(bar):
    return bar.metoda(1)

foo4(promenna)


## 3. Modul typing

Modul `typing` je knihovna zavedená v Pythonu 3.5, která rozšiřuje možnosti type hinting o další specializované typy:
- List s předepsaným typemm prvků (např. `List[int]`)
- Předepsaná délka kontejneru (např. `List[int, 3]`, nebo `List[int, int, int]`)

In [ ]:
from typing import List, Tuple, Dict, Union, Optional

# List s definovaným typem
seznam_celych_cisel: List[int] = [1, 2, 3]

# Tuple s definovaným typem a počtem prvků
ntice_celych_cisel: Tuple[int, int, int] = (1, 2, 3)

# Dict s definovaným typem klíče a hodnoty
slovnik_celych_cisel: Dict[str, int] = {"jedna": 1, "dva": 2, "tri": 3}


### Kombinace typů

`Union` a `Optional` jsou dva užitečné typy, které poskytuje modul `typing`. Níže je podrobnější popis těchto dvou typů.



#### Union

`Union` se používá, když je třeba vyjádřit, že hodnota může být jedním z více možných typů. Typ `Union` vytváří sjednocení (union) mezi zadanými typy a naznačuje, že hodnota může být jakýmkoli z těchto typů.



In [ ]:
from typing import Union


def vypis_hodnotu(hodnota: Union[int, str]) -> None:
    print(f"Hodnota je: {hodnota}")


vypis_hodnotu(42)  # Hodnota je: 42
vypis_hodnotu("Ahoj")  # Hodnota je: Ahoj


#### Optional
`Optional` je zvláštní případ `Union`, který se používá, když hodnota může být daným typem nebo None. V podstatě `Optional[Typ]` je ekvivalent `Union[Typ, None]`.

In [ ]:
from typing import Optional, List

def najdi_index(hledany_prvek: str, seznam_prvku: List[str]) -> Optional[int]:
    if hledany_prvek in seznam_prvku:
        return seznam_prvku.index(hledany_prvek)
    else:
        return None

prvky = ["jablko", "hruska", "banan"]
index_jablka = najdi_index("jablko", prvky)  # index_jablka = 0
index_pomeranc = najdi_index("pomeranc", prvky)  # index_pomeranc = None


#### Nová syntaxe od Pythonu 3.10
V Pythonu 3.10 byla přidána nová syntaxe pro `Union` a `Optional`. Nová syntaxe umožňuje použít `|` místo `Union` a `None` místo `Optional`.

In [ ]:
# Příklad s Union
from typing import List


def vypis_hodnotu(hodnota: int | str) -> None:
    print(f"Hodnota je: {hodnota}")


vypis_hodnotu(42)  # Hodnota je: 42
vypis_hodnotu("Ahoj")  # Hodnota je: Ahoj

# Příklad s Optional


def najdi_index(hledany_prvek: str, seznam_prvku: List[str]) -> int | None:
    if hledany_prvek in seznam_prvku:
        return seznam_prvku.index(hledany_prvek)
    else:
        return None


prvky = ["jablko", "hruska", "banan"]
index_jablka = najdi_index("jablko", prvky)  # index_jablka = 0
index_pomeranc = najdi_index("pomeranc", prvky)  # index_pomeranc = None


### Splecifické hodnoty `Literal`

`Literal` je typ, který umožňuje definovat konkrétní hodnoty, které může mít proměnná. `Literal` je užitečný, pokud chceme definovat konkrétní hodnoty, které může mít proměnná, například pro výběr z možností v rozbalovacím menu. 

In [ ]:
from typing import Literal


def set_status(status: Literal['pending', 'approved', 'rejected']) -> None:
    print(f"Setting status to: {status}")


# This will work
set_status('approved')

# This will raise a type error during static type checking
set_status('unknown')


### Anotace typů pro dodatečnou dokumentace `Annotated`

Slouží k přidání dodatečných informací k anotaci typu bez toho abychom měnili typ. Funkčnost `Annotated` je podobná jako kdybychom tyto informace přidali jako docstring.
 

In [ ]:
from typing import Annotated, List

# Define a custom annotation


def max_length(length: int) -> int:
    return length


def process_names(names: Annotated[List[str], max_length(5)]) -> None:
    for name in names:
        print(f"Processing name: {name}")


names = ["Alice", "Bob", "Charlie", "David", "Eve"]
process_names(names)

names2 = ["Alice", "Bob", "Charlie", "David", "Eve", "Frank"]
process_names(names2)

## 4. Type aliasy
Type aliasy lze použít k vytvoření čitelnějších type hintů:

In [ ]:
from typing import List, Tuple

Souradnice = Tuple[float, float]
Cesta = List[Souradnice]


def vypocet_vzdalenosti(cesta: Cesta) -> float:
    vzdalenost = 0
    for i, souradnice in enumerate(cesta):
        if i == 0:
            continue
        x1, y1 = cesta[i - 1]
        x2, y2 = souradnice
        vzdalenost += ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
    return vzdalenost

cesta = [(0.1, 0.1), (1.1, 1.1), (2.1, 2.1)]
vypocet_vzdalenosti(cesta)


## 5. Generika
Generika jsou užitečná pokud chceme pouze specifikovat, že v některých částech kódu má být stejný typ, ale je nám už jedno jaký typ přesně to bude.

In [ ]:
from typing import List, TypeVar

T = TypeVar("T")


def najdi_prvni(prvky: List[T], cil: T) -> int:
    for i, prvek in enumerate(prvky):
        if prvek == cil:
            return i
    return -1

prvky = [1, 2, 3]
najdi_prvni(prvky, 2)

prvky = ["jablko", "hruska", "banan"]
najdi_prvni(prvky, "hruska")


In [ ]:
prvky = [1.3, 2.4, 3.5]
najdi_prvni(prvky, "3")


U generických typů můžeme specifikovat seznam typů, které může generický typ obsahovat.

In [ ]:
from typing import TypeVar

OmezenyTyp = TypeVar("OmezenyTyp", int, str)


def najdi_prvni(prvky: List[OmezenyTyp], cil: OmezenyTyp) -> int:
    for i, prvek in enumerate(prvky):
        if prvek == cil:
            return i
    return -1

prvky = [1, 2, 3]
najdi_prvni(prvky, 2)

prvky = ["jablko", "hruska", "banan"]
najdi_prvni(prvky, "hruska")


In [ ]:

prvky = [1.3, 2.4, 3.5]
najdi_prvni(prvky, 3.5)



## 6. Type hinty pro NumPy

Type hinting v NumPy je trochu složitější. Vestavěné nástroje Pythonu umožňují pouze kontrolu, zda se jedná o objekt `ndarray`. My bychom však chtěli, aby se kontrola typů u `ndarray` prováděla na úrovni vlastností pole, například `ndarray.shape` nebo `ndarray.dtype`.

Částečně nám s tímto může pomoci modul `numpy.typing`, který umožňuje definovat type hinty pro datový typ obsažený v `ndarray`.


In [ ]:
import numpy as np
from numpy.typing import NDArray


def process_array(arr: NDArray[np.float64]) -> None:
    print(arr)


input_array: NDArray[np.float64] = np.array([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]
], dtype=np.float64)  # Explicitně nastavíme dtype na float64

process_array(input_array)

In [ ]:
input_array = np.array([1, 2, 3], dtype=np.int8)
process_array(input_array)


Bohužel však typování pro rozměry pole nebo alespoň dimenzi pole není plně podporováno.

## 7. Type hinty při běhu

Type hinty sice nejsou přímo vyhodnocovány, ale můžeme se na ně podívat pomocí `__annotations__` atributu třídy.

In [ ]:
def foo(par: int, par2: str) -> None:
    print(par, par2)

print(foo.__annotations__)

## 8. Typování před definicí třídy

Často chceme otypovat metodu třídy (např. `__add__`) typem této třídy. Toto bohužel nelze standardní cestou. 

Řešením, je napsat typ do uvozovek.

In [ ]:
class LepsiCislo:
    def __init__(self, hodnota: int | float) -> None:
        self.hodnota = hodnota

    def __add__(self, other: "LepsiCislo") -> "LepsiCislo":
        return LepsiCislo(self.hodnota + other.hodnota)